In [1]:
import mne
import os
from tqdm import tqdm_notebook
import neurokit2 as nk
import numpy as np
import yasa
from scipy.signal import welch

In [ ]:
ratings = pd.read_excel("ratings/Result movies.xlsx")

In [ ]:
n_subjects = 21
matrix = pd.DataFrame()
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    filtered = mne.io.read_raw(
        "raw_filtered_fif/S{}_filtered_raw.fif".format(
            "0" + str(Subject) if Subject < 10 else Subject
        )
    )
    raw = mne.io.read_raw(
        "raw_full/"
        + (
            ("00" + str(Subject) if Subject < 10 else "0" + str(Subject)) + "a"
            if Subject == 2 or Subject == 5
            else ("00" + str(Subject) if Subject < 10 else "0" + str(Subject))
        )
        + ".vhdr"
    )
    annot = filtered.annotations
    n_events = 12 if Subject == 11 else 16
    for i in tqdm_notebook(range(0, n_events, 2)):
        film = annot[i + 1]["description"].split("_")[-1]
        try:
            watch = raw.copy().crop(
                annot[i + 1]["onset"], annot[i + 1]["onset"] + annot[i + 1]["duration"]
            )
        except Exception as e:
            watch = raw.copy().crop(
                annot[i + 1]["onset"],
                annot[i + 1]["onset"] + annot[i + 1]["duration"] - 0.01,
            )
        df, info = nk.eda_process(watch["EDA"][0][0], sampling_rate=1000)
        amplitudes = []
        for i in range(min(len(info["SCR_Peaks"]), len(info["SCR_Onsets"]))):
            if str(info["SCR_Onsets"][i]) == "nan":
                continue
            amp = (
                watch["EDA"][0][0][int(info["SCR_Peaks"][i])]
                - watch["EDA"][0][0][int(info["SCR_Onsets"][i])]
            )
            amplitudes.append(amp)
        dataframe = {
            "EDA": np.sum(amplitudes) / (watch.last_samp / 1000),
            "Film": film,
            "Subject": Subject,
            "Labels": ratings.loc[Subject - 1][film + ".mp4"],
        }
        pd.DataFrame(dataframe, index=[0]).to_excel(
            "EDA_final/{}_{}.xlsx".format(Subject, film)
        )
        matrix = pd.concat([matrix, pd.DataFrame(dataframe, index=[0])], axis=0)